https://www.imagetracking.org.uk/2018/03/piping-numpy-arrays-to-other-processes-in-python/

In [1]:
import os

In [2]:
import subprocess

In [3]:
import numpy as np

In [4]:
import pickle

In [5]:
lmp_py_str = """import numpy as np
from mpi4py import MPI
import pickle
import sys


if MPI.COMM_WORLD.rank == 0:
    data = sys.stdin.buffer.read(8)
    dlen = int.from_bytes(data, byteorder='big')
    data = pickle.loads(sys.stdin.buffer.read(dlen))
else:
    data = None

data = MPI.COMM_WORLD.bcast(data, root=0)

if MPI.COMM_WORLD.rank == 1:
    print(len(data))
    
MPI.COMM_WORLD.Barrier()
MPI.Finalize()
"""

In [6]:
with open('pipe.py', 'w') as f:
    f.writelines(lmp_py_str)

In [7]:
process = subprocess.Popen(
    ["mpiexec", "-n", "2", "python", "pipe.py"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    stdin=subprocess.PIPE,
    bufsize=1,
)

In [8]:
data = np.random.random(8171)

In [9]:
data = np.random.random(8172) 

In [10]:
data = pickle.dumps(data)

In [11]:
data_len = len(data).to_bytes(8, byteorder='big')
process.stdin.write(data_len)
process.stdin.write(data)
process.stdin.flush()

In [12]:
process.stdout.read()

b''